<a href="https://colab.research.google.com/github/crossmodalritual/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
import operator
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from google.colab import drive  # Importa o módulo drive

# Constantes e configurações
DRIVE_MOUNT_PATH = "/content/drive"  # Caminho padrão de montagem do Google Drive
GRAPHPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "graficos")  # Caminho completo no Drive
SUMMARYPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive", "sumarios")  # Caminho completo no Drive
DEFAULT_PALETTE = "Spectral"
FIGSIZE = (10, 6)

# Configuração de estilo para gráficos
sns.set(style="whitegrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'legend.facecolor': 'black',
    'axes.labelcolor': 'lime',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'figure.titlesize': 20,
    'axes.titlesize': 16,
    'axes.labelsize': 12,
})

# Função para carregar os dados
def load_data(csv_content):
    try:
        csv_file = io.StringIO(csv_content)
        df = pd.read_csv(csv_file, skiprows=1, names=['Idade', 'Modelo', 'Sexo'])
        return df
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None

def preprocess_numeric_cols(df, cols):
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    return df.dropna(subset=cols)

def save_fig(filename):
    filepath = os.path.join(GRAPHPATH, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()


def create_figure():
    return plt.figure(figsize=FIGSIZE)


# Funções para gerar gráficos (adaptadas para lidar com strings e números)
def generate_violinplot(df, x, y, hue, title, filename, split=False, palette=None):
    create_figure()
    if palette is None:
        palette = DEFAULT_PALETTE

    # Garante que x e hue sejam numéricos ou categóricos válidos
    df_plot = df.copy()
    if not pd.api.types.is_numeric_dtype(df_plot[x]):
        df_plot[x] = df_plot[x].astype('category').cat.codes
    if hue is not None and not pd.api.types.is_numeric_dtype(df_plot[hue]):
        df_plot[hue] = df_plot[hue].astype('category').cat.codes

    # Garante que o número de cores na paleta seja suficiente
    if hue is not None:
        num_hues = df_plot[hue].nunique()
        if len(palette) < num_hues:
            palette = sns.color_palette(palette, n_colors=num_hues)

    sns.violinplot(x=x, y=y, hue=hue, data=df_plot, split=split, palette=palette)
    plt.title(title)
    save_fig(filename)

def generate_histplot(df, x, hue, title, filename, bins=20, palette=None):
    create_figure()
    if palette is None:
        palette = DEFAULT_PALETTE

    df_plot = df.copy()
    if not pd.api.types.is_numeric_dtype(df_plot[x]):
        df_plot[x] = df_plot[x].astype('category').cat.codes
    if hue is not None and not pd.api.types.is_numeric_dtype(df_plot[hue]):
        df_plot[hue] = df_plot[hue].astype('category').cat.codes

    if hue is not None:
        num_hues = df_plot[hue].nunique()
        if len(palette) < num_hues:
            palette = sns.color_palette(palette, n_colors=num_hues)

    sns.histplot(data=df_plot, x=x, hue=hue, bins=bins, palette=palette, kde=True)
    plt.title(title)
    save_fig(filename)

def generate_kdeplot(df, x, hue, title, filename, palette=None):
    create_figure()
    if palette is None:
        palette = DEFAULT_PALETTE

    df_plot = df.copy()
    if not pd.api.types.is_numeric_dtype(df_plot[x]):
        df_plot[x] = df_plot[x].astype('category').cat.codes
    if hue is not None and not pd.api.types.is_numeric_dtype(df_plot[hue]):
        df_plot[hue] = df_plot[hue].astype('category').cat.codes

    if hue is not None:
        num_hues = df_plot[hue].nunique()
        if len(palette) < num_hues:
            palette = sns.color_palette(palette, n_colors=num_hues)

    sns.kdeplot(data=df_plot, x=x, hue=hue, palette=palette, fill=True)
    plt.title(title)
    save_fig(filename)



# Funções para gerar gráficos de regressões não lineares
def generate_polynomial_regression(df, x, y, degree, title, filename):
    df_reg = df.copy()
    # Converte x e y para numérico, lidando com erros
    for col in [x, y]:
        if not pd.api.types.is_numeric_dtype(df_reg[col]):
            try:
                df_reg[col] = pd.to_numeric(df_reg[col], errors='raise') #Força erro
            except (ValueError, TypeError) as e:
                print(f"Erro: A coluna '{col}' não pôde ser convertida para numérica. Pulando regressão. Detalhes: {e}")
                return  # Sai da função se a conversão falhar

    df_reg.dropna(subset=[x, y], inplace=True) # Remove NaNs, *após* conversão

    x_data = df_reg[x].values.reshape(-1, 1)
    y_data = df_reg[y].values.reshape(-1, 1)

    if len(x_data) == 0 or len(y_data) == 0: #Verifica se há dados
        print(f"Erro: Não há dados suficientes para a regressão após limpeza. Pulando.")
        return

    polynomial_features = PolynomialFeatures(degree=degree)
    x_poly = polynomial_features.fit_transform(x_data)

    model = LinearRegression()
    model.fit(x_poly, y_data)

    y_poly_pred = model.predict(x_poly)

    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(x_data, y_poly_pred), key=sort_axis)
    x_data, y_poly_pred = zip(*sorted_zip)

    create_figure() #Cria a figura *antes* de plotar
    plt.scatter(df_reg[x], df_reg[y], color='cyan', label="Dados")
    plt.plot(x_data, y_poly_pred, color='magenta', label=f"Regressão (Grau {degree})")
    plt.title(title, color='white')
    plt.xlabel(x, color='lime')
    plt.ylabel(y, color='lime')
    plt.legend()
    save_fig(filename)

def generate_logarithmic_regression(df, x, y, title, filename):
    df_reg = df.copy()
    for col in [x, y]:
        if not pd.api.types.is_numeric_dtype(df_reg[col]):
            try:
                df_reg[col] = pd.to_numeric(df_reg[col], errors='raise')
            except (ValueError, TypeError) as e:
                print(f"Erro: A coluna '{col}' não é numérica e não pode ser convertida. Pulando regressão. Detalhes: {e}")
                return

    df_reg.dropna(subset=[x, y], inplace=True)
    df_reg = df_reg[df_reg[x] > 0]

    if df_reg.empty:
        print("Erro: Não há dados válidos para regressão logarítmica após limpeza. Pulando.")
        return

    x_data = np.log(df_reg[x]).values.reshape(-1, 1)
    y_data = df_reg[y].values.reshape(-1, 1)

    model = LinearRegression()
    model.fit(x_data, y_data)
    y_log_pred = model.predict(x_data)

    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(df_reg[x], np.exp(y_log_pred)), key=sort_axis)
    x_data_original, y_log_pred = zip(*sorted_zip)

    create_figure()
    plt.scatter(df_reg[x], df_reg[y], color='cyan', label="Dados")
    plt.plot(x_data_original, y_log_pred, color='magenta', label="Regressão Logarítmica")
    plt.title(title, color='white')
    plt.xlabel(x, color='lime')
    plt.ylabel(y, color='lime')
    plt.legend()
    save_fig(filename)

def generate_exponential_regression(df, x, y, title, filename):
    df_reg = df.copy()
    for col in [x, y]:
        if not pd.api.types.is_numeric_dtype(df_reg[col]):
            try:
                df_reg[col] = pd.to_numeric(df_reg[col], errors='raise')
            except (ValueError, TypeError) as e:
                print(f"Erro: A coluna '{col}' não é numérica e não pode ser convertida. Pulando regressão. Detalhes: {e}")
                return

    df_reg.dropna(subset=[x, y], inplace=True)
    df_reg = df_reg[df_reg[y] > 0]

    if df_reg.empty:
        print("Erro: Não há dados válidos para regressão exponencial após limpeza. Pulando.")
        return

    x_data = df_reg[x].values.reshape(-1, 1)
    y_data = np.log(df_reg[y]).values.reshape(-1, 1)

    model = LinearRegression()
    model.fit(x_data, y_data)
    y_exp_pred = np.exp(model.predict(x_data))

    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(x_data, y_exp_pred), key=sort_axis)
    x_data, y_exp_pred = zip(*sorted_zip)

    create_figure()
    plt.scatter(df_reg[x], df_reg[y], color='cyan', label="Dados")
    plt.plot(x_data, y_exp_pred, color='magenta', label="Regressão Exponencial")
    plt.title(title, color='white')
    plt.xlabel(x, color='lime')
    plt.ylabel(y, color='lime')
    plt.legend()
    save_fig(filename)

# Função para gerar sumário estatístico detalhado (adaptada)
def generate_statistical_summary_detailed(df, y_var, filename="sumario_detalhado.txt"):
    try:
        # Cria variáveis numéricas para todas as colunas relevantes
        df_summary = df.copy()
        for col in ['Modelo', 'Sexo', 'Idade']:
            if not pd.api.types.is_numeric_dtype(df_summary[col]):
                df_summary[col] = df_summary[col].astype('category').cat.codes

        X = df_summary[['Idade', 'Sexo', 'Modelo']]  # Usa as versões numéricas
        X = sm.add_constant(X)
        y = df_summary[y_var]

        model = sm.OLS(y, X).fit()
        summary = model.summary()

        filepath = os.path.join(SUMMARYPATH, filename)  # Caminho completo
        with open(filepath, 'w') as f:
            f.write(summary.as_text())

    except Exception as e:
        print(f"Erro ao gerar o sumário estatístico detalhado: {e}")

# Função principal (main) - Adaptada
if __name__ == "__main__":
    # Monta o Google Drive
    drive.mount(DRIVE_MOUNT_PATH, force_remount=True)

    # Cria os diretórios no Google Drive, se não existirem
    os.makedirs(GRAPHPATH, exist_ok=True)
    os.makedirs(SUMMARYPATH, exist_ok=True)

    # Dados de exemplo
    csv_data = """
,Idade,Modelo,Sexo
0,58,Ford,Masculino
1,28,Fiat,Feminino
2,23,Ford,Masculino
3,45,Chevrolet,Masculino
4,56,Chevrolet,Feminino
5,41,Ford,Masculino
6,41,Chevrolet,Feminino
7,49,Fiat,Masculino
8,30,Chevrolet,Feminino
9,56,Ford,Masculino
10,42,Fiat,Feminino
11,45,Chevrolet,Masculino
12,49,Ford,Masculino
13,41,Chevrolet,Feminino
14,29,Fiat,Masculino
15,44,Chevrolet,Feminino
16,45,Ford,Masculino
17,41,Chevrolet,Feminino
18,29,Fiat,Masculino
19,57,Chevrolet,Feminino
20,25,Ford,Masculino
"""

    df = load_data(csv_data)

    if df is not None:
        # --- Geração dos Gráficos ---
        paleta1 = ["#00FF00", "#FFA500"]
        paleta2 = ["#00FF00", "#0000FF", "#FFA500"]
        paleta3 = ["#00FF00", "#0000FF", "#FFA500", "#FF00FF"]

        # Certifique-se de que 'Idade' seja numérica para os gráficos de regressão
        df['Idade'] = pd.to_numeric(df['Idade'], errors='coerce')
        df.dropna(subset=['Idade'], inplace=True)  # Remove linhas com 'Idade' NaN

        # Cria colunas numéricas temporárias para os gráficos, se necessário
        df_graficos = df.copy()
        if not pd.api.types.is_numeric_dtype(df_graficos['Modelo']):
            df_graficos['Modelo_Num'] = df_graficos['Modelo'].astype('category').cat.codes
        if not pd.api.types.is_numeric_dtype(df_graficos['Sexo']):
            df_graficos['Sexo_Num'] = df_graficos['Sexo'].astype('category').cat.codes

        # Usa as colunas numéricas nos gráficos
        generate_violinplot(df_graficos, 'Modelo_Num', 'Idade', 'Sexo_Num', 'Idade por Modelo e Sexo (Bipartido)', 'violin_idade_modelo_sexo_split.png', split=True, palette=paleta1)
        generate_violinplot(df_graficos, 'Sexo_Num', 'Idade', 'Modelo_Num', 'Idade por Sexo e Modelo (Bipartido)', 'violin_idade_sexo_modelo_split.png', split=True, palette=paleta2)
        generate_violinplot(df_graficos, 'Modelo_Num', 'Idade', 'Sexo_Num', 'Idade por Modelo e Sexo', 'violin_idade_modelo_sexo.png', palette=paleta3)
        generate_violinplot(df_graficos, 'Sexo_Num', 'Idade', 'Modelo_Num', 'Idade por Sexo e Modelo', 'violin_idade_sexo_modelo.png', palette=paleta1)

        generate_histplot(df_graficos, 'Idade', 'Sexo_Num', 'Distribuição de Idade por Sexo', 'hist_idade_sexo.png', bins=20, palette=paleta2)
        generate_histplot(df_graficos, 'Idade', None, 'Distribuição de Idade', 'hist_idade_geral.png', bins=20, palette=paleta3)

        generate_kdeplot(df_graficos, 'Idade', 'Modelo_Num', 'Densidade de Idade por Modelo', 'kde_idade_modelo.png', palette=paleta1)
        generate_kdeplot(df_graficos, 'Idade', 'Sexo_Num', 'Densidade de Idade por Sexo', 'kde_idade_sexo.png', palette=paleta2)

        try:
            #Regressões, usando as colunas numéricas criadas
            generate_polynomial_regression(df_graficos, 'Idade', 'Sexo_Num', 2, 'Regressão Polinomial (Grau 2) - Idade vs. Sexo', 'poly_idade_sexo_2.png')
            generate_polynomial_regression(df_graficos, 'Idade', 'Modelo_Num', 3, 'Regressão Polinomial (Grau 3) - Idade vs. Modelo', 'poly_idade_modelo_3.png')
            generate_logarithmic_regression(df_graficos, 'Idade', 'Sexo_Num', 'Regressão Logarítmica - Idade vs. Sexo', 'log_idade_sexo.png')
            generate_exponential_regression(df_graficos, 'Idade', 'Modelo_Num', 'Regressão Exponencial - Idade vs. Modelo', 'exp_idade_modelo.png')

        except ValueError as ve:
            print(f"Erro ao gerar regressão: {ve}. Verifique se os dados são adequados para o tipo de regressão.")
        except Exception as e:
            print(f"Erro inesperado ao gerar regressão: {e}")

        generate_statistical_summary_detailed(df_graficos, 'Idade')  # Usa o df_graficos para o sumário

        print("Gráficos e sumário gerados com sucesso!")

Mounted at /content/drive


<ipython-input-17-625a957cacc1>:81: UserWarning: The palette list has more values (4) than needed (2), which may not be intended.
  sns.violinplot(x=x, y=y, hue=hue, data=df_plot, split=split, palette=palette)
<ipython-input-17-625a957cacc1>:101: UserWarning: The palette list has more values (3) than needed (2), which may not be intended.
  sns.histplot(data=df_plot, x=x, hue=hue, bins=bins, palette=palette, kde=True)
<ipython-input-17-625a957cacc1>:101: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.histplot(data=df_plot, x=x, hue=hue, bins=bins, palette=palette, kde=True)
<ipython-input-17-625a957cacc1>:121: UserWarning: The palette list has more values (3) than needed (2), which may not be intended.
  sns.kdeplot(data=df_plot, x=x, hue=hue, palette=palette, fill=True)


Gráficos e sumário gerados com sucesso!
